In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
from scipy.stats import norm, multivariate_normal

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

## Hamiltonian Monte Carlo

HMC requires five things:

1. The negative log-probability $U$ of the data in the current position.

$$
   U = -\left ( \sum_i \log p(y_i \mid \mu_y, 1) + \sum_i \log p(x_i \mid \mu_x, 1) 
            + \log p(\mu_y \mid 0, 0.5) + \log p(\mu_x \mid 0, 0.5)\right )
$$

In the current setting, $p(x \mid a, b)$ is the pdf of the Normal distribution with mean $a$ and standard deviation $b$.

### Code 9.5

In [2]:
def calc_U(x, y, q, a=0, b=1.0, c=0, d=1.0):
    mu_x, mu_y = q
    
    U = np.sum(norm.logpdf(y, loc=mu_y, scale=1) + 
               norm.logpdf(x, loc=mu_x, scale=1) + 
               norm.logpdf(mu_y, loc=a, scale=b) + 
               norm.logpdf(mu_x, loc=c, scale=d))
    return -U

2. A gradient function that returns the gradient of the negative log-probability of the data in the current position. 

Since $p(x \mid a, b)$ is the pdf of the Normal distribution with mean $a$ and standard deviation $b$, we have:
$$
    \log p(y \mid a, b) = \log \frac{1}{\sqrt{2 \pi} b} - \frac{1}{2 b^2} (y - a)^2.
$$
The partial derivative wrt $a$ is:
$$
    \frac{\partial \log p(y \mid a, b)}{\partial a} = \frac{y - a}{b^2}.
$$
The partial derivative wrt $y$ is:
$$
    \frac{\partial \log p(y \mid a, b)}{\partial y} = - \frac{y - a}{b^2}.
$$

Therefore $\partial U / \partial \mu_x$ is given by:
$$
    \frac{\partial U}{\partial \mu_x} = 
        \sum_i \frac{\partial p(x_i \mid \mu_x, 1)}{\partial \mu_x} + 
        \frac{\partial p(\mu_x \mid 0, 0.5)}{\partial \mu_x} =
        \sum_i \frac{(x_i - \mu_x)}{1^2} - \frac{\mu_x - 0}{0.5^2}.
$$

Similarly, $\partial U / \partial \mu_y$ is given by:
$$
    \frac{\partial U}{\partial \mu_y} = 
        \sum_i \frac{\partial p(y_i \mid \mu_y, 1)}{\partial \mu_y} + 
        \frac{\partial p(\mu_y \mid 0, 0.5)}{\partial \mu_y} =
        \sum_i \frac{(y_i - \mu_y)}{1^2} - \frac{\mu_y - 0}{0.5^2}.
$$

In [3]:
def grad_U(x, y, q, a=0, b=1.0, c=0, d=1.0):
    mu_x = q[0]
    mu_y = q[1]

    grad_mu_x = np.sum(x - mu_x) - (mu_x - a) / b**2
    grad_mu_y = np.sum(y - mu_y) - (mu_y - c) / d**2
    
    return np.array([grad_mu_x, grad_mu_y])

In [4]:
grad_U(x=np.array([1, 2]), y=np.array([3, 4]), q=np.array([0.1, 0.2]))

array([2.7, 6.4])

3. a step-size `epsilon`

4. a count of leapfrog steps $L$

5. a starting position `current_q`

### HMC2 Function Code 9.8, 9.9, 9.10

Credit: https://github.com/pymc-devs/resources/blob/master/Rethinking_2/Chp_09.ipynb

In [21]:
from typing import Callable


def HMC2(U: Callable, grad_U: Callable, epsilon: float, L: int, current_q: np.array, x: np.array, y: np.array):
    """
        This function runs a single trajectory and produces one sample. To build a chain of samples, 
        call this function repeatedly.
    """
    q = current_q
    # random flick - p is momentum
    p = np.random.normal(loc=0, scale=1, size=len(q))  
    current_p = p
    
    # Make a half step for momentum at the beginning
    p = p - 0.5 * epsilon * grad_U(x, y, q) 
    
    # initialize bookkeeping - saves trajectory
    q_traj = np.full(shape=(L + 1, len(q)), fill_value=np.nan)
    p_traj = q_traj.copy()
    
    q_traj[0, :] = current_q
    p_traj[0, :] = p

    # Code 9.9 starts here
    for i in range(L):
        q += epsilon * p  # Full step for the position
        q_traj[i + 1, :] = q
        
        # Make a full step for the momentum, except at the end of trajectory
        if i != L - 1:
            p -= epsilon * grad_U(x, y, q)
            p_traj[i + 1, :] = p

    # Make a half step for momentum at the end
    p = p - 0.5 * epsilon * grad_U(x, y, q)
    p_traj[L, :] = p
    
    # Negate momentum at end of trajectory to make the proposal symmetric
    p = -1 * p
    
    # Evaluate potential and kinetic energies at start and end of trajectory
    current_U = U(x, y, current_q)
    # mass = 1 unit so that KE = 0.5 * p**2
    current_K = np.sum(current_p ** 2) / 2.0
    proposed_U = U(x, y, q)
    proposed_K = np.sum(p ** 2) / 2
    
    # Accept or reject the state at end of trajectory, returning either
    # the position at the end of the trajectory or the initial position
    accept = False
    print(current_U - proposed_U + current_K - proposed_K)
    if np.log(np.random.uniform()) < np.log(current_U - proposed_U + current_K - proposed_K):
        new_q = q  # accept
        accept = True
    else:
        new_q = current_q  # reject

    return dict(q=new_q, traj=q_traj, p_traj=p_traj, accept=accept)

### Code 9.7

Credit: https://github.com/pymc-devs/resources/blob/master/Rethinking_2/Chp_09.ipynb

In [22]:
np.random.seed(42)

# test data
real = multivariate_normal([0, 0], np.identity(2))
x, y = real.rvs(50).T

Q = {}
Q["q"] = np.array([-0.1, 0.2])
pr = 0.5
step = 0.03
# L = 11  # 0.03 / 28 for U-turns -- 11 for working example
n_samples = 4
_, axs = plt.subplots(1, 2, figsize=[8, 6], constrained_layout=True)

for L, ax in zip([11, 28], axs):
    ax.scatter(*Q["q"], color="k", marker="x", zorder=3)
    if L == 11:
        ax.text(*Q["q"] + 0.015, "start", weight="bold")
    for i in range(n_samples):
        Q = HMC2(calc_U, grad_U, step, L, Q["q"], x, y)
        ax.scatter(*Q["q"], color="w", marker="o", edgecolor="k", lw=2, zorder=3)
        if n_samples < 10:
            for j in range(L):
                K0 = np.sum(Q["p_traj"][j, :] ** 2) / 2  # kinetic energy
                ax.plot(
                    Q["traj"][j : j + 2, 0],
                    Q["traj"][j : j + 2, 1],
                    color="k",
                    lw=1 + 1 * K0,
                    alpha=0.3,
                    zorder=1,
                )
            ax.scatter(*Q["traj"].T, facecolor="w", edgecolor="gray", lw=1, zorder=2, s=10)
            if L == 11:
                ax.text(*Q["q"] + [0.02, -0.03], f"{i + 1}", weight="bold")

    ax.set_title(f"2D Gaussian, L = {L}")
    ax.set_xlabel("mux")
    ax.set_ylabel("muy")

    # draw background contours based on real probability defined above
    ax.set_xlim(-pr, pr)
    ax.set_ylim(-pr, pr)
    xs, ys = np.mgrid[-pr:pr:0.01, -pr:pr:0.01]
    p = real.logpdf(np.vstack([xs.flat, ys.flat]).T).reshape(xs.shape)
    ax.contour(xs, ys, p, 4, colors=[(0, 0, 0, 0.3)])
    ax.set_aspect(1);

-49.683602617024334
-1288.4005244484447
-35014.11976104163
-975291.6927664067
-40250241124.37018
-1596944709767191.2
-6.335964899962168e+19
-2.5138285819715136e+24


<ipython-input-21-c9646402b695>:52: RuntimeWarning: invalid value encountered in log
  if np.log(np.random.uniform()) < np.log(current_U - proposed_U + current_K - proposed_K):
<ipython-input-21-c9646402b695>:52: RuntimeWarning: invalid value encountered in log
  if np.log(np.random.uniform()) < np.log(current_U - proposed_U + current_K - proposed_K):
<ipython-input-21-c9646402b695>:52: RuntimeWarning: invalid value encountered in log
  if np.log(np.random.uniform()) < np.log(current_U - proposed_U + current_K - proposed_K):
<ipython-input-21-c9646402b695>:52: RuntimeWarning: invalid value encountered in log
  if np.log(np.random.uniform()) < np.log(current_U - proposed_U + current_K - proposed_K):
<ipython-input-21-c9646402b695>:52: RuntimeWarning: invalid value encountered in log
  if np.log(np.random.uniform()) < np.log(current_U - proposed_U + current_K - proposed_K):
<ipython-input-21-c9646402b695>:52: RuntimeWarning: invalid value encountered in log
  if np.log(np.random.uniform(